# Identify Contrails with Keras

In [1]:
# reinstall tensorflow-io
# to avoid the UserWarning: unable to load libtensorflow_io_plugins.so

#!pip install tensorflow-io

In [14]:
# ==============================

TRAIN = True
#PLATFORM = 'kaggle'
PLATFORM = 'gcp'

# ==============================

In [15]:
#%cp /kaggle/input/identify-contrails/contrails_2023-07-15_15-10-49.h5 .
#%mv weights_v8_contrails-unet_cst-lr.h5 contrails-unet_cst-lr.h5

if not TRAIN:
    checkpoint_path = 'contrails_2023-07-15_15-10-49.h5'

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import datetime
import math
import pathlib
import random
import shutil

from pytz import timezone

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
PRINT_TIME_FORMAT = "%Y-%m-%d %H:%M:%S %Z%z"
FILE_TIME_FORMAT = "%Y-%m-%d_%H-%M-%S"

In [18]:
now_time = datetime.datetime.now(timezone('CET'))

file_time_str = now_time.strftime(FILE_TIME_FORMAT)

print('Started', now_time.strftime(PRINT_TIME_FORMAT))

Started 2023-07-19 18:35:58 CEST+0200


In [19]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as backend

In [20]:
tf.__version__

'2.11.0'

In [22]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num CPUs Available:  1
Num GPUs Available:  1


In [23]:
%pwd

'/home/jupyter/identify-contrails/notebooks'

In [24]:
#---------------------------------------------------------------------------79

Following U-Net model is adapted from:
- https://keras.io/examples/vision/oxford_pets_image_segmentation
- https://www.kaggle.com/code/shashwatraman/simple-unet-baseline-train-lb-0-580

# Setup

In [25]:
if PLATFORM == 'kaggle':

    WORK_DIR = '/kaggle/working'  # preserved if notebook is saved
    TEMP_DIR = '/kaggle/temp'  # just during current session

    DATA_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'

elif PLATFORM == 'gcp':

    WORK_DIR = '/home/jupyter/kaggle/working'  # preserved if notebook is saved
    TEMP_DIR = '/home/jupyter/kaggle/temp'  # just during current session

    DATA_DIR = '/home/jupyter/kaggle/input/google-research-identify-contrails-reduce-global-warming'
    
    %cd $WORK_DIR

/home/jupyter/kaggle/working


In [52]:
class Paths:
    train = os.path.join(DATA_DIR, 'train')
    valid = os.path.join(DATA_DIR, 'validation')
    test = os.path.join(DATA_DIR, 'test')

In [53]:
train_ids = sorted(os.listdir(Paths.train))
valid_ids = sorted(os.listdir(Paths.valid))
test_ids = sorted(os.listdir(Paths.test))
print(len(train_ids), len(valid_ids), len(test_ids))

20529 1856 2


In [54]:
class ABI:
    bands = {name: idx for idx, name in enumerate([
        '08', '09', '10', '11', '12', '13', '14', '15', '16'])}
    colors = {name: idx for idx, name in enumerate([
        'red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'black'])}

In [55]:
N_TIMES_BEFORE = 4
N_TIMES_AFTER = 3

In [56]:
def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def get_ash_colors(sample_id, split_dir):
    """
    Based on bands: 11, 14, 15
    
    Args:
        sample_id(str): The id of the example i.e. '1000216489776414077'
        split_dir(str): The split directoryu i.e. 'test', 'train', 'val'
    """
    band15 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_15.npy")
    band14 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_14.npy")
    band11 = np.load(DATA_DIR + f"/{split_dir}/{sample_id}/band_11.npy")

    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    ash_colors = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    
    return ash_colors

In [57]:
def get_individual_mask(sample_id, split_dir):
    masks_path = DATA_DIR + f"/{split_dir}/{sample_id}/human_individual_masks.npy"
    pixel_mask = np.load(masks_path)
    return pixel_mask

In [58]:
def get_pixel_mask(sample_id, split_dir):
    masks_path = DATA_DIR + f"/{split_dir}/{sample_id}/human_pixel_masks.npy"
    pixel_mask = np.load(masks_path)
    return pixel_mask

### Check some values (DEVEL)

In [59]:
sample_id = train_ids[3]

ash_colors = get_ash_colors(sample_id, 'train')[..., N_TIMES_BEFORE]

print(ash_colors.shape)
for color in range(3):
    array = ash_colors[..., color]
    print(array.min(), array.max())

(256, 256, 3)
0.0 0.6714986
0.0 0.8929291
0.0 0.8980291


In [60]:
pixel_mask = get_pixel_mask(sample_id, 'train')

print(pixel_mask.shape)
print(pixel_mask.min(), pixel_mask.max())

(256, 256, 1)
0 0


# Model

In [61]:
SEED = 42

In [62]:
class Config:
    
    img_size = (256, 256)
    
    num_epochs = 10  # <DEVEL> else 10
    num_classes = 1
    batch_size = 16  # <DEVEL> else 32
    
    threshold = 0.40
    
    seed = SEED

In [63]:
# https://keras.io/examples/keras_recipes/reproducibility_recipes/

# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) `tensorflow` random seed
# 3) `python` random seed
keras.utils.set_random_seed(Config.seed)

# See also:
# tf.config.experimental.enable_op_determinism()

Model inspired by:
- https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/
- file:///D:/Courses/2023-07_Advanced-Computer-Vision-with-TensorFlow/lecture-notes/C3_W3_Image-Segmentation.pdf

In [64]:
def conv2d_block(input_tensor, n_filters, kernel_size=3):
    x = input_tensor
    for i in range(2):
        x = tf.keras.layers.SeparableConv2D(
            filters = n_filters, kernel_size=(kernel_size, kernel_size), padding='same')(x)
        #? kernel_initializer = 'he_normal'
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
    return x

def encoder_block(inputs, n_filters, pool_size, dropout):
    f = conv2d_block(inputs, n_filters=n_filters)
    p = tf.keras.layers.MaxPooling2D(pool_size)(f)
    p = tf.keras.layers.Dropout(dropout)(p)
    return f, p

def encoder(inputs, dropout=0.1):
    f1, p1 = encoder_block(inputs, n_filters=64, pool_size=(2,2), dropout=dropout)
    f2, p2 = encoder_block(p1, n_filters=128, pool_size=(2,2), dropout=dropout)
    f3, p3 = encoder_block(p2, n_filters=256, pool_size=(2,2), dropout=dropout)
    f4, p4 = encoder_block(p3, n_filters=512, pool_size=(2,2), dropout=dropout)
    return p4, (f1, f2, f3, f4)

def bottleneck(inputs):
    bottle_neck = conv2d_block(inputs, n_filters=1024)
    return bottle_neck

def decoder_block(inputs, conv_output, n_filters, kernel_size, strides, dropout):
    u = tf.keras.layers.Conv2DTranspose(
        n_filters, kernel_size, strides=strides, padding = 'same')(inputs)
    u = tf.keras.layers.BatchNormalization()(u)
    c = tf.keras.layers.concatenate([u, conv_output])
    c = tf.keras.layers.Dropout(dropout)(c)
    c = conv2d_block(c, n_filters, kernel_size=3)
    return c

def decoder(inputs, convs, num_classes, dropout=0.1):
    f1, f2, f3, f4 = convs
    c6 = decoder_block(inputs, f4, n_filters=512, kernel_size=(3,3), strides=(2,2), dropout=dropout)
    c7 = decoder_block(c6, f3, n_filters=256, kernel_size=(3,3), strides=(2,2), dropout=dropout)
    c8 = decoder_block(c7, f2, n_filters=128, kernel_size=(3,3), strides=(2,2), dropout=dropout)
    c9 = decoder_block(c8, f1, n_filters=64, kernel_size=(3,3), strides=(2,2), dropout=dropout)
    if num_classes == 1:
        activation = "sigmoid"
    else:
        activation = "softmax"
    outputs = layers.Conv2D(num_classes, kernel_size=3, activation=activation, padding="same")(c9)
    return outputs

def unet(image_size, num_classes):
    inputs = tf.keras.layers.Input(shape=(image_size,image_size,3))
    encoder_output, convs = encoder(inputs)
    #model = tf.keras.Model(inputs=inputs, outputs=encoder_output)  # debug
    bottle_neck = bottleneck(encoder_output)
    outputs = decoder(bottle_neck, convs, num_classes)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [65]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
#model = get_model(Config.img_size, Config.num_classes)
model = unet(image_size=256, num_classes=1)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 separable_conv2d (SeparableCon  (None, 256, 256, 64  283        ['input_1[0][0]']                
 v2D)                           )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['separable_conv2d[0][0]']       
 alization)                     )                                                             

# Prepare datasets

In [66]:
N_SAMPLES = None  # None to take all
N_PARTIAL = 128  # 128

In [67]:
class AshColorSingleFrames(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, sample_ids, split_dir, n_samples=None):
        self.batch_size = batch_size
        self.img_size = img_size
        self.split_dir = split_dir
        self.sample_ids = sample_ids[:n_samples]

    def __len__(self):
        return math.ceil(len(self.sample_ids) / self.batch_size)

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_sample_ids = self.sample_ids[i : i + self.batch_size]
        
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, sample_id in enumerate(batch_sample_ids):
            img = get_ash_colors(sample_id, self.split_dir)
            x[j] = img[..., N_TIMES_BEFORE]

        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        if self.split_dir != 'test':
            for j, sample_id in enumerate(batch_sample_ids):
                img = get_pixel_mask(sample_id, self.split_dir)
                y[j] = img
        
        return x, y

In [68]:
train_set = AshColorSingleFrames(Config.batch_size, Config.img_size, train_ids, 'train',
                                 n_samples=N_SAMPLES)  # <DEVEL>
print('number of batches:', len(train_set))

number of batches: 1284


In [69]:
valid_set = AshColorSingleFrames(Config.batch_size, Config.img_size, valid_ids, 'validation',
                                 n_samples=N_SAMPLES)  # <DEVEL>
print('number of batches:', len(valid_set))

number of batches: 116


In [70]:
partial_set = AshColorSingleFrames(Config.batch_size, Config.img_size, valid_ids, 'validation',
                                 n_samples=N_PARTIAL)  # <DEVEL>
print('number of batches:', len(partial_set))

number of batches: 8


In [71]:
test_set = AshColorSingleFrames(Config.batch_size, Config.img_size, test_ids, 'test')
print('number of batches:', len(test_set))

number of batches: 1


Check batch dimensions (x, y):

In [72]:
train_set[0][0].shape, train_set[0][1].shape

((16, 256, 256, 3), (16, 256, 256, 1))

# Training

`dice_coef` adapted from:
- https://stackoverflow.com/questions/72195156/correct-implementation-of-dice-loss-in-tensorflow-keras
- https://www.kaggle.com/code/shashwatraman/simple-unet-baseline-train-lb-0-580

In [73]:
def dice_coef(y_true, y_pred, smooth=0.001, threshold=None):
    y_true_f = backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = backend.flatten(tf.cast(y_pred, tf.float32))
    # ValueError: No gradients provided for any variable
    if threshold is not None:
        y_pred_f = backend.flatten(
            tf.cast(tf.math.greater(tf.cast(y_pred, tf.float32), threshold), tf.float32))
    intersection = backend.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (backend.sum(y_true_f) + backend.sum(y_pred_f) + smooth)
    return dice

def threshold_dice_coef(y_true, y_pred, smooth=0.001):
    return dice_coef(y_true, y_pred, smooth=smooth, threshold=Config.threshold)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

Check `dice_coef()` on one of the samples.

In [74]:
sample_id = train_ids[3]

merged_mask = get_pixel_mask(sample_id, 'train')
indiv_masks = get_individual_mask(sample_id, 'train')

print(dice_coef(tf.convert_to_tensor(merged_mask),
                tf.convert_to_tensor(merged_mask)))
for idv in range(6):
    print(dice_coef(tf.convert_to_tensor(merged_mask),
                    tf.convert_to_tensor(indiv_masks[..., idv])))

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


Learning rate scheduler:
- https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules
- https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay - warmup only from v2.13.1 on

In [75]:
if TRAIN:
    checkpoint_path = f"contrails_{file_time_str}.h5"

print(f'checkpoint file: {checkpoint_path}')

checkpoint file: contrails_2023-07-19_18-35-58.h5


In [76]:
initial_learning_rate = 0.01
decay_steps = len(train_set)
decay_rate = 0.7

cos_scheduler = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps)

exp_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate)

In [77]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=[dice_coef])
#model.compile(optimizer="adam", loss=dice_loss, metrics=[dice_coef])

callbacks = [
    #keras.callbacks.LearningRateScheduler(exp_scheduler),
    keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True)
]

In [ ]:
if TRAIN:
    # Train the model, doing validation at the end of each epoch.
    model.fit(train_set, epochs=Config.num_epochs, validation_data=valid_set, callbacks=callbacks,
             workers=4, use_multiprocessing=True)
else:
    # Loads the weights
    model.load_weights(checkpoint_path)

Before submitting, we will have to apply the **threshold**!

In [ ]:
def apply_threshold(pred, threshold):
    return (pred > threshold).astype(np.int32)

In [ ]:
#keras.backend.clear_session()

# Evaluate

In [ ]:
if not TRAIN:    
    # Evaluate the model

    ALL_BATCHES = True
    BATCH_IDX = 0
    SAMPLE_IDX = 1

    if ALL_BATCHES:
        loss, acc = model.evaluate(partial_set, verbose=2)
    else:
        eval_images, eval_masks = partial_set[BATCH_IDX]
        loss, acc = model.evaluate(eval_images, eval_masks, verbose=2)

    print("Saved model, accuracy: {:5.2f}%".format(100 * acc))

    predictions = model.predict(partial_set)

In [ ]:
def eval_dice_coef(sample_set, pred_set, batch_size, threshold):
    dice_coef_per_batch = np.full(len(sample_set), np.nan)
    for idx in range(len(sample_set)):
        x, y = sample_set[idx]
        pred = pred_set[idx*batch_size:(idx + 1)*batch_size]
        _coef = dice_coef(y, pred, threshold=threshold)
        dice_coef_per_batch[idx] = _coef
    return dice_coef_per_batch

if not TRAIN:
    _coefs = eval_dice_coef(
        partial_set, predictions, batch_size=partial_set.batch_size, threshold=None)
    print(f'w/o threshod: {_coefs.mean():.2%}')

    _coefs = eval_dice_coef(
        partial_set, predictions, batch_size=partial_set.batch_size, threshold=Config.threshold)
    print(f'{Config.threshold}: {_coefs.mean():.2%}')

In [ ]:
def plot_prediction(img, truth, pred):

    fig, axs = plt.subplots(1, 4, figsize=(16, 8))

    axs[0].imshow(img)
    axs[0].set_title("Ash Color Image")

    axs[1].imshow(truth)
    axs[1].set_title("Ground Truth")

    axs[2].imshow(pred)
    axs[2].set_title("Prediction")

    axs[3].imshow(img)
    axs[3].imshow(truth, cmap='Reds', alpha=.3, interpolation='none')
    axs[3].set_title('Contrail mask on ash color image')

    plt.tight_layout() 
    plt.show()

    return

if not TRAIN:
    eval_images, eval_masks = partial_set[BATCH_IDX]
    idx = SAMPLE_IDX
    threshold = Config.threshold
    plot_prediction(
        eval_images[idx], eval_masks[idx], apply_threshold(predictions[idx], threshold))

## Make predictions on test dataset

In [ ]:
predictions = model.predict(test_set)

In [ ]:
len(predictions)

## Create a submission

In [ ]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s


In [ ]:
test_recs = os.listdir(os.path.join(DATA_DIR, 'test'))

In [ ]:
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'), index_col='record_id')[0:0]

for test_id, pred in zip(test_ids, predictions):
    
    mask = apply_threshold(pred, Config.threshold)
    
    # notice the we're converting rec to an `int` here:
    submission.loc[int(test_id), 'encoded_pixels'] = list_to_string(rle_encode(mask))
    
submission.to_csv('submission.csv')

In [ ]:
print('Terminated', datetime.datetime.now(timezone('CET')).strftime(PRINT_TIME_FORMAT))